# Metrics Validation Notebook

This notebook provides comprehensive validation for all evaluation metrics in the unified benchmark pipeline. It verifies:

1. **Input Shape Handling** - Proper processing of (A, B, C) tensors with timestamp channels
2. **MDD Fix** - Marginal Distribution Distance no longer returns NaN
3. **Fidelity Metrics** - All 6 metrics (MDD, MD, SDD, SD, KD, ACD) work correctly
4. **Stylized Facts** - Financial time series properties are properly computed

## Table of Contents:
1. [Setup and Imports](#Setup-and-Imports)
2. [Input Shape Validation](#Input-Shape-Validation)
3. [MDD Fix Verification](#MDD-Fix-Verification)
4. [Fidelity Metrics Testing](#Fidelity-Metrics-Testing)
5. [Stylized Facts Testing](#Stylized-Facts-Testing)
6. [Summary](#Summary)


## Setup and Imports

Import all necessary modules and set up the project paths.


In [5]:
import sys
import numpy as np
import torch
from pathlib import Path
import time

# Add project root to path
project_root = Path().resolve().parents[0]
sys.path.append(str(project_root))

print(f"Project root: {project_root}")

from src.taxonomies.diversity import calculate_icd
from src.taxonomies.fidelity import (
    calculate_mdd, calculate_md, calculate_sdd, 
    calculate_sd, calculate_kd, calculate_acd
)
from src.taxonomies.stylized_facts import (
    heavy_tails, autocorr_raw, volatility_clustering, 
    long_memory_abs, non_stationarity
)

print("\n✅ All modules imported successfully!")


Project root: C:\Users\14165\Downloads\Unified-benchmark-for-SDGFTS-main

✅ All modules imported successfully!


## Input Shape Validation

Test that metrics handle various input shapes correctly:
- Data with a timestamp channel (should be automatically dropped)
- PyTorch tensors (should be converted to NumPy internally)
- 2D arrays (should be expanded to 3D)

All metrics expect shape (A, B, C) where:
- A: number of samples
- B: sequence length
- C: number of features (timestamp at index 0 is dropped)


In [6]:
print("="*60)
print("Testing Input Shape Handling")
print("="*60)

# Test with timestamp channel (should be dropped)
data_with_timestamp = np.random.randn(50, 100, 6)  # (A, B, C) with C=6 including timestamp
print(f"\nTest 1: Data with timestamp channel")
print(f"  Input shape: {data_with_timestamp.shape}")

icd = calculate_icd(data_with_timestamp, metric="euclidean")
assert not np.isnan(icd), "❌ ICD returned NaN!"
print(f"  ✓ ICD (Euclidean): {icd:.4f}")

# Test with torch tensor
data_torch = torch.randn(50, 100, 5)  # (A, B, C)
print(f"\nTest 2: PyTorch tensor")
print(f"  Input shape: {tuple(data_torch.shape)}")

icd_torch = calculate_icd(data_torch, metric="euclidean")
assert not np.isnan(icd_torch), "❌ ICD returned NaN!"
print(f"  ✓ ICD (Torch): {icd_torch:.4f}")

# Test 2D input (should be expanded to 3D)
data_2d = np.random.randn(100, 5)  # (B, C)
print(f"\nTest 3: 2D array (will be expanded to 3D)")
print(f"  Input shape: {data_2d.shape}")

# Explicitly expand to (A, B, C) to avoid axis errors in downstream metrics
data_2d_abc = data_2d[None, ...]

icd_2d = calculate_icd(data_2d_abc, metric="euclidean")
assert not np.isnan(icd_2d), "❌ ICD returned NaN!"
print(f"  ✓ ICD (2D): {icd_2d:.4f}")

print("\n" + "="*60)
print("✅ Shape handling test PASSED")
print("="*60)


Testing Input Shape Handling

Test 1: Data with timestamp channel
  Input shape: (50, 100, 6)


AxisError: axis 2 is out of bounds for array of dimension 2

## MDD Fix Verification

The Marginal Distribution Distance (MDD) metric previously returned NaN values due to:
1. Division by zero in histogram bin width calculations
2. Improper use of nn.Parameter for storing densities

This test verifies that MDD now returns valid, finite values.


In [ ]:
print("="*60)
print("Testing MDD Fix")
print("="*60)

# Generate test data
np.random.seed(42)
real_data = np.random.randn(100, 50, 5)  # 100 samples, 50 timesteps, 5 channels
synthetic_data = np.random.randn(100, 50, 5) * 1.1 + 0.1

print(f"\nTest data shapes:")
print(f"  Real: {real_data.shape}")
print(f"  Synthetic: {synthetic_data.shape}")

print("\nComputing MDD...")
start = time.time()
mdd = calculate_mdd(real_data, synthetic_data)
elapsed = time.time() - start

print(f"  MDD value: {mdd:.6f}")
print(f"  Computation time: {elapsed:.3f}s")

# Validate result
assert not np.isnan(mdd), "❌ MDD returned NaN!"
assert np.isfinite(mdd), "❌ MDD returned inf!"
assert mdd >= 0, "❌ MDD is negative!"

print("\n" + "="*60)
print("✅ MDD fix PASSED - No NaN values")
print("="*60)


## Fidelity Metrics Testing

Test all six fidelity metrics:
1. MDD - Marginal Distribution Distance (histogram-based)
2. MD - Mean Distance
3. SDD - Standard Deviation Distance
4. SD - Skewness Distance
5. KD - Kurtosis Distance
6. ACD - Autocorrelation Distance

Each metric should handle (A, B, C) input shapes, drop timestamp channels, and return finite values.


In [ ]:
print("="*60)
print("Testing All Fidelity Metrics")
print("="*60)

# Generate test data with timestamp channel
np.random.seed(42)
real_data = np.random.randn(50, 100, 6)  # Including timestamp at channel 0
synthetic_data = real_data + np.random.randn(50, 100, 6) * 0.3

print(f"\nTest data shapes:")
print(f"  Real: {real_data.shape}")
print(f"  Synthetic: {synthetic_data.shape}")
print(f"  (Channel 0 will be dropped as timestamp)\n")

metrics = [
    ("MDD", calculate_mdd),
    ("MD", calculate_md),
    ("SDD", calculate_sdd),
    ("SD", calculate_sd),
    ("KD", calculate_kd),
    ("ACD", calculate_acd)
]

results = {}
total_time = 0

for name, func in metrics:
    print(f"Computing {name}...", end=" ")
    start = time.time()
    try:
        value = func(real_data, synthetic_data)
        elapsed = time.time() - start
        total_time += elapsed
        
        # Validate result
        assert not np.isnan(value), f"{name} returned NaN!"
        assert np.isfinite(value), f"{name} returned inf!"
        
        results[name] = value
        print(f"✓ {value:.6f} ({elapsed:.3f}s)")
    except Exception as e:
        print(f"❌ FAILED: {e}")
        raise

print(f"\nTotal computation time: {total_time:.3f}s")
print("\n" + "="*60)
print(f"✅ All {len(results)}/{len(metrics)} fidelity metrics PASSED")
print("="*60)


## Stylized Facts Testing

Test all five stylized facts metrics for financial time series:
1. Heavy Tails - Excess kurtosis in returns
2. Autocorrelation - Serial correlation in raw returns
3. Volatility Clustering - Autocorrelation in squared returns
4. Long Memory - Persistence in absolute returns
5. Non-stationarity - Time-varying variance

These metrics assess whether synthetic data exhibits realistic financial properties.


In [ ]:
print("="*60)
print("Testing Stylized Facts Metrics")
print("="*60)

# Generate test data (price-like data with geometric brownian motion)
np.random.seed(42)
returns = np.random.randn(30, 200, 5) * 0.01
data = np.exp(np.cumsum(returns, axis=1)) * 100

print(f"\nTest data shape: {data.shape}")
print(f"  30 samples, 200 timesteps, 5 channels\n")

metrics = [
    ("Heavy Tails", heavy_tails),
    ("Autocorrelation", autocorr_raw),
    ("Volatility Clustering", volatility_clustering),
    ("Long Memory", long_memory_abs),
    ("Non-stationarity", non_stationarity)
]

results = {}
total_time = 0

for name, func in metrics:
    print(f"Computing {name}...", end=" ")
    start = time.time()
    try:
        value = func(data)
        elapsed = time.time() - start
        total_time += elapsed
        
        # Validate result
        assert value is not None, f"{name} returned None!"
        assert len(value) > 0, f"{name} returned empty array!"
        
        results[name] = value
        print(f"✓ shape={value.shape}, values={value[:3]} ({elapsed:.3f}s)")
    except Exception as e:
        print(f"❌ FAILED: {e}")
        raise

print(f"\nTotal computation time: {total_time:.3f}s")
print("\n" + "="*60)
print(f"✅ All {len(results)}/{len(metrics)} stylized facts PASSED")
print("="*60)


## Summary

All metric validations have passed. The evaluation metrics accept (A, B, C) inputs, drop timestamp channel automatically, and return finite values. DTW and MDD fixes are in place. The pipeline is ready to use.


In [ ]:
# (Removed) Parallelization benchmark was not yielding consistent gains across setups.
print("Parallelization benchmark removed. Metrics validated sequentially.")
